# Data Analysis of the generated images and residuals without outliers

In this notebook, the data-set is generated with lenstronomy. The aim is to show the data that will be used for the neural network and confirm if the generated data fits our expectations with simulated images.
As we deal with and absolute error on the residuals, some percentage of error are tested to make sure that the error is difficult to see with the naked eye. 
Finally the distribution of variable and labels are checked in order to make sure that our dataset building is correct.

Final configuration for the simulated images:
Lens' mass variation :
- Model : Power-law Elliptical Mass Distribution

>$\kappa = \frac{3-\gamma }{2}\left ( \frac{\theta_E}{\sqrt{q e_1^2+ e_2^2/q}} \right )^{\gamma-1}$
 
* Einstein radius: $\theta_E \in \mathcal{N}_{log}(\mu:0,\sigma:0.1)$

* Power law slope: $\gamma \in \mathcal{N}_{log}(\mu:0.7,\sigma:0.1)$
* Ellipsity* : $e_1 \in \mathcal{N}(\mu :0, \sigma : 0.2)$   and $e_2 \in \mathcal{N}(\mu :0, \sigma : 0.2)$
* Center : $x = 0$ and $y = 0$

Source variation :
* Model : Sersic ellipse
* Amplitude : $amp \in  \mathcal{U}(20, 24)$
* Sersic radius : $R_{sersic} \in  \mathcal{N}_{log}(\mu:-0.7,\sigma:0.4)$
* Sersic index : $n_{sersic} \in  \mathcal{N}_{log}(\mu:0.7,\sigma:0.4)$
* Ellipsity* : $e_1 \in \mathcal{U}(\mu :0, \sigma : 0.2)$   and $e_2 \in \mathcal{U}(\mu :0, \sigma : 0.2)$
* Center : $x \in \mathcal{U}(-0.5,0.5)$ and $y \in \mathcal{U}(-0.5,0.5)$

Note* : The ellipsity range is determined and is approximated with the following equations, where $q \in \mathcal{U}(0.7, 1)$ and $\phi \in \mathcal{U}(0, \frac{\pi}{2})$

>$e_1 = \frac{1-q}{1+q}\cos{(2\phi)}$     and     $e_2 = \frac{1-q}{1+q}\sin{(2\phi)} $
    
The added error is proportionnally defined such that each quantities is defined as following :

>$A_err = A \pm p A$

Where $A$ is the current quantity value and $p$ the user defined percentage of error. 

The final data configuration parameters :
* Size of the whole dataset: $s = 6000$
* Ratio of error maps: $r = 75\%$
* Percentage of error : $p = \begin{bmatrix}5\% & 1.5\% & 5\%\end{bmatrix}$

In this data set outliers are not considered, which are noise-like error residual maps and maps with a maximal absolute amplitude superior to $6$. The bound to remove the noise-like data is by computing the Peak Signal to-Noise Ratio (PSNR) between a map that do not contain any noise and any errors and comparing it to the mean PSNR of maps that only contain noise. If the values is lower, the image is automatically discard, otherwise, the value is kept.

Mean square error :
>$MSE = \frac{1}{NM}\sum_{n= 0}^N \sum_{m=0}^M $

Peak Signal to-Noise Ratio : 
>$PSNR = $



### 0. Import

In [1]:
import numpy as np
import pandas as pd
from statistics import mean
from scipy.stats import chi2, chi, gaussian_kde

from helpers.data_generation.file_management import*
from helpers.data_generation.error_generation_chi2 import*

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
sns.set(style="whitegrid")

import warnings
warnings.filterwarnings("ignore")

### 1. Build the data set

The residual maps are generated and stored in the repository `data/dataset/` under the name `S[size]R[ratio]_lens.h5`, while the metadata is saved under the name `S[size]R[ratio]_meta.h5`. The `[size]` correspond to the number of images and the `[ratio]` is the percentage of error.

In [ ]:
# Build the four classes 
config_repo_model = 'data/configFile/config_model'
size = 600; ratio = 0.75;
percent = np.array([0.005, 0.015, 0.005])
res = Residual()
res.build(size, ratio = ratio, per_error = percent)

print('Data Generation Finished')

In [ ]:
str_ID =  "S"+str(size)+"R"+str(int(ratio*100))
[final_array, metadata] = read_hdf5(str_ID)

print('Reading data Finished')

index = metadata.index

indices_noerror = index[[col == [0,0] for col in metadata['class']]]
indices_mass = index[[col == [1,0] for col in metadata['class']]]
indices_source = index[[col == [0,1] for col in metadata['class']]]
indices_masssource = index[[col == [1,1] for col in metadata['class']]]

## 2. Plot the residual maps and simulated images

In [ ]:
f, axes = plt.subplots(9, 9, figsize=(36, 30), sharex=False, sharey=False)

for count, k in enumerate(np.array([0, 3, 6])):
    for i in range(0,9):
        pos1 = axes[k,i].imshow(final_array[indices_mass[i+9*count],0,:,:], vmin=-6, vmax=6, origin='lower',cmap=plt.cm.BuPu_r)
        pos2 = axes[k+1,i].imshow(final_array[indices_source[i+9*count],0,:,:], vmin=-6, vmax=6, origin='lower',cmap=plt.cm.BuPu_r)
        pos3 = axes[k+2,i].imshow(final_array[indices_masssource[i+9*count],0,:,:], vmin=-6, vmax=6, origin='lower',cmap=plt.cm.BuPu_r)

        axes[k,i].set_yticklabels([]); axes[k,i].set_xticklabels([])
        axes[k+1,i].set_yticklabels([]); axes[k+1,i].set_xticklabels([])
        axes[k+2,i].set_yticklabels([]); axes[k+2,i].set_xticklabels([])
        f.colorbar(pos1, ax=axes[k,i]); f.colorbar(pos2, ax=axes[k+1,i])
        f.colorbar(pos3, ax=axes[k+2,i])
    
    
    pad = 5
    font = 30

    axes[k,0].annotate('Mass error', xy=(0, 0.5), xytext=(-axes[k,0].yaxis.labelpad - pad, 0),
                       xycoords=axes[k,0].yaxis.label, textcoords='offset points', ha='right', va='center',fontsize=font)
    axes[k+1,0].annotate('Source error', xy=(0, 0.5), xytext=(-axes[k+1,0].yaxis.labelpad - pad, 0),
                       xycoords=axes[k+1,0].yaxis.label, textcoords='offset points', ha='right', va='center',fontsize=font)
    axes[k+2,0].annotate('Mass &\nsource error', xy=(0, 0.5), xytext=(-axes[k+2,0].yaxis.labelpad - pad, 0),
                       xycoords=axes[k+2,0].yaxis.label, textcoords='offset points', ha='right', va='center',fontsize=font)
    
f.suptitle('Lenstronomy residual map with {:.1f}% of mass error and {:.1f}% of source error'.format(percent[0]*100, percent[1]*100), size = 'large',fontsize=font)
plt.tight_layout(rect=[0, 0.02, 1, 0.97])
plt.show()

plt.show()
f.savefig('figures/datanalysis/'+str(int(percent[0]*100))+'percenterror_chi2.jpeg')

In [ ]:
from scipy import signal
cor = signal.correlate2d (final_array[indices_mass[0],:,:], final_array[indices_mass[0],:,:])

f, axes = plt.subplots(8, 8, figsize=(48, 30), sharex=False, sharey=False)
ran = 6
for count, k in enumerate(np.array([0, 4,])):
    for i in range(0,8):
        cor_mass = signal.correlate2d (final_array[indices_mass[i+9*count],0,:,:], final_array[indices_mass[i+9*count],0,:,:])
        cor_source = signal.correlate2d (final_array[indices_source[i+9*count],0,:,:], final_array[indices_source[i+9*count],0,:,:])
        cor_sourcemass = signal.correlate2d (final_array[indices_masssource[i+9*count],0,:,:], final_array[indices_masssource[i+9*count],0,:,:])
        cor_noerror = signal.correlate2d (final_array[indices_noerror[i+9*count],0,:,:], final_array[indices_noerror[i+9*count],0,:,:])
        cor_mass = (cor_mass- np.mean(cor_mass))/np.std(cor_mass)
        cor_source = (cor_source- np.mean(cor_source))/np.std(cor_source)
        cor_sourcemass = (cor_sourcemass- np.mean(cor_sourcemass))/np.std(cor_sourcemass)
        cor_noerror = (cor_noerror- np.mean(cor_noerror))/np.std(cor_noerror)
        pos1 = axes[k,i].imshow(cor_mass,  vmin=-ran, vmax=ran, origin='lower',cmap=plt.cm.BuPu_r)
        pos2 = axes[k+1,i].imshow(cor_source,  vmin=-ran, vmax=ran, origin='lower',cmap=plt.cm.BuPu_r)
        pos3 = axes[k+2,i].imshow(cor_sourcemass, vmin=-ran, vmax=ran, origin='lower',cmap=plt.cm.BuPu_r)
        pos4 = axes[k+3,i].imshow(cor_noerror, vmin=-ran, vmax=ran, origin='lower',cmap=plt.cm.BuPu_r)

        axes[k,i].set_yticklabels([]); axes[k,i].set_xticklabels([])
        axes[k+1,i].set_yticklabels([]); axes[k+1,i].set_xticklabels([])
        axes[k+2,i].set_yticklabels([]); axes[k+2,i].set_xticklabels([])
        axes[k+3,i].set_yticklabels([]); axes[k+3,i].set_xticklabels([])
        f.colorbar(pos1, ax=axes[k,i]); f.colorbar(pos2, ax=axes[k+1,i]); f.colorbar(pos3, ax=axes[k+2,i]); f.colorbar(pos4, ax=axes[k+3,i])
    
    
    pad = 5
    font = 30

    axes[k,0].annotate('Mass error', xy=(0, 0.5), xytext=(-axes[k,0].yaxis.labelpad - pad, 0),
                       xycoords=axes[k,0].yaxis.label, textcoords='offset points', ha='right', va='center',fontsize=font)
    axes[k+1,0].annotate('Source error', xy=(0, 0.5), xytext=(-axes[k+1,0].yaxis.labelpad - pad, 0),
                       xycoords=axes[k+1,0].yaxis.label, textcoords='offset points', ha='right', va='center',fontsize=font)
    axes[k+2,0].annotate('Mass &\nsource error', xy=(0, 0.5), xytext=(-axes[k+2,0].yaxis.labelpad - pad, 0),
                       xycoords=axes[k+2,0].yaxis.label, textcoords='offset points', ha='right', va='center',fontsize=font)
    
    axes[k+3,0].annotate('No error', xy=(0, 0.5), xytext=(-axes[k+3,0].yaxis.labelpad - pad, 0),
                       xycoords=axes[k+3,0].yaxis.label, textcoords='offset points', ha='right', va='center',fontsize=font)
f.suptitle('Lenstronomy residual map correlation', size = 'large')
plt.tight_layout(rect=[0, 0.02, 1, 0.97])
plt.show()
f.savefig('figures/datanalysis/Autocorrelation_chi2.jpeg')

In [ ]:
f, axes = plt.subplots(1, 3, figsize=(10, 3), sharex=False, sharey=False)
index = metadata.index

indices_noerror = index[[col == [0,0] for col in metadata['class']]]

for i in range(3):
    pos = axes[i].imshow(final_array[indices_noerror[i],0,:,:], vmin=-6, vmax=6, origin='lower',cmap=plt.cm.BuPu_r)

    axes[i].set_yticklabels([]); axes[i].set_xticklabels([])
    f.colorbar(pos, ax=axes[i]);
    
pad = 5

axes[0].annotate('No error', xy=(0, 0.5), xytext=(-axes[0].yaxis.labelpad - pad, 0),
                   xycoords=axes[0].yaxis.label, textcoords='offset points', ha='right', va='center')

f.suptitle('Lenstronomy residual map with no error', size = 'large')
plt.tight_layout()
plt.show()

plt.show()
f.savefig('figures/datanalysis/Noerror_chi2.jpeg')

In [ ]:
f, axes = plt.subplots(9, 9, figsize=(36, 30), sharex=False, sharey=False)
dataset_model = LensDataset(size = size, percent = percent)
p_decal = 10

for i in range(0,9):
    for k in range(9):
        
        image_real = dataset_model.images[i+k*p_decal][0] + dataset_model.image_config.noise_for_model(model = dataset_model.images[i][0])
        pos1 = axes[k,i].imshow(image_real, cmap = 'viridis', origin='lower')
        axes[k,i].set_yticklabels([]); axes[k,i].set_xticklabels([])
        f.colorbar(pos1, ax=axes[k,i])

f.suptitle('Simulated images', size = 'large')
f.tight_layout()
plt.show()

plt.show()
f.savefig('figures/datanalysis/simulated_chi2.jpeg')

## 3. Plot distributions

### 3.1 Label distribution

In [ ]:
meta_dist = metadata.copy(deep=True)
meta_dist.loc[[col == [0,0] for col in meta_dist['class']], 'class'] = 'NoErrors'
meta_dist.loc[[col == [1,0] for col in meta_dist['class']], 'class'] = 'Mass'
meta_dist.loc[[col == [0,1] for col in meta_dist['class']], 'class'] = 'Source'
meta_dist.loc[[col == [1,1] for col in meta_dist['class']], 'class'] = 'Mass&Source'

ax = sns.countplot(meta_dist['class'],label="Count")
fig = ax.get_figure()

fig.savefig('figures/datanalysis/balancelabel_chi2.jpeg')

### 3.2 Parameters distribution according to label

In [ ]:
meta_err = res.metadata_error.copy(deep=True)

meta_err.loc[[col == [0,0] for col in meta_err['class']], 'class'] = 'NoMass'
meta_err.loc[[col == [1,0] for col in meta_err['class']], 'class'] = 'Mass'
meta_err.loc[[col == [0,1] for col in meta_err['class']], 'class'] = 'NoMass'
meta_err.loc[[col == [1,1] for col in meta_err['class']], 'class'] = 'Mass'

cols = meta_err.columns[2:6].tolist()
cols.append('class')
g = sns.pairplot(meta_err[cols], diag_kind="kde", corner = True, hue="class", palette = "magma")
g.map_lower(sns.scatterplot, s=5)
g.map_lower(sns.histplot, bins=300, pthresh=.1)
g.map_lower(sns.kdeplot, levels=5, color="w", linewidths=1)
g.savefig('figures/datanalysis/statMass_chi2.jpeg')

In [ ]:
meta_err = res.metadata_error.copy(deep=True)

meta_err.loc[[col == [0,0] for col in meta_err['class']], 'class'] = 'NoSource'
meta_err.loc[[col == [1,0] for col in meta_err['class']], 'class'] = 'NoSource'
meta_err.loc[[col == [0,1] for col in meta_err['class']], 'class'] = 'Source'
meta_err.loc[[col == [1,1] for col in meta_err['class']], 'class'] = 'Source'
meta_err=meta_err.drop(columns=['percent'])


cols = meta_err.columns[7:].tolist()
g = sns.pairplot(meta_err[cols], diag_kind="kde", corner = True, hue="class", palette = "magma")
g.map_lower(sns.scatterplot, s=5)
g.map_lower(sns.histplot, bins=300, pthresh=.1)
g.map_lower(sns.kdeplot, levels=5, color="w", linewidths=1)
g.savefig('figures/datanalysis/statSource_chi2.jpeg')

### 3.3 Distribution of the distance between a image with an error & an image without error

In [ ]:
sns.axes_style("whitegrid")
dist_mass = [np.sum((final_array[indices_mass[i],0,:,:]-final_array[indices_noerror[i],0,:,:])**2)/64**2 for i in range(len(indices_noerror))]
dist_source = [np.sum((final_array[indices_source[i],0,:,:]-final_array[indices_noerror[i],0,:,:])**2)/64**2 for i in range(len(indices_noerror))]
dist_masssource = [np.sum((final_array[indices_masssource[i],0,:,:]-final_array[indices_noerror[i],0,:,:])**2)/64**2 for i in range(len(indices_noerror))]

g = sns.kdeplot(dist_mass, label = 'Mass error', bw_adjust=.1)
sns.kdeplot(dist_source, label = 'Source error', bw_adjust=.1)
sns.kdeplot(dist_masssource, label = 'Mass & source error', bw_adjust=.1)
plt.xlim([1.5, 5])
plt.legend()
plt.savefig('figures/datanalysis/distancefinal_chi2.jpeg')
plt.show()


print('Mean distance of the mass : '+ str(mean(dist_mass))+ '\nMean distance of the source : ' + str(mean(dist_source))+ 
      '\nMean distance of the mass & the source : ' +  str(mean(dist_masssource)))

### 3.4 Distribution of the Chi2 test

In [ ]:
from scipy.stats import chi2, chi

ki2_noerror = [np.sum(final_array[i,0,:,:]**2)/(final_array.shape[3]*final_array.shape[2])for i in indices_noerror]
ki2_mass = [np.sum(final_array[i,0,:,:]**2)/(final_array.shape[3]*final_array.shape[2])for i in indices_mass]
ki2_source = [np.sum(final_array[i,0,:,:]**2)/(final_array.shape[3]*final_array.shape[2])for i in indices_source]
ki2_masssource = [np.sum(final_array[i,0,:,:]**2)/(final_array.shape[3]*final_array.shape[2])for i in indices_masssource]

print('Chi2 mean of the mass : '+ str(mean(ki2_mass))+ '\nChi2 mean of the source : ' + str(mean(ki2_source))+ 
      '\nChi2 mean of the mass & the source : ' +  str(mean(ki2_masssource))+ '\nChi2 mean of no error : ' + str(mean(ki2_noerror)))

sns.kdeplot(ki2_mass, label = 'Mass error', bw_adjust=.1)
sns.kdeplot(ki2_source, label = 'Source error', bw_adjust=.1)
sns.kdeplot(ki2_masssource, label = 'Mass & source error', bw_adjust=.1)
plt.xlim([0.5, 4])
plt.legend()
plt.savefig('figures/datanalysis/Chi2final_chi2.jpeg')
plt.show()

In [ ]:
rms_noise = np.asarray(res.rms_noise)
noise_final = np.asarray([final_array[i,0,:,:] - 3*rms_noise[i,:,:] for i in range(size)])
noise_final = noise_final.clip(min=0)

ki2_noerror = [np.sum(noise_final[i,0,:,:]**2)/(final_array.shape[3]*final_array.shape[2])for i in indices_noerror]
ki2_mass = [np.sum(noise_final[i,0,:,:]**2)/(final_array.shape[3]*final_array.shape[2])for i in indices_mass]
ki2_source = [np.sum(noise_final[i,0,:,:]**2)/(final_array.shape[3]*final_array.shape[2])for i in indices_source]
ki2_masssource = [np.sum(noise_final[i,0,:,:]**2)/(final_array.shape[3]*final_array.shape[2])for i in indices_masssource]

print('Chi2 mean of the mass : '+ str(mean(ki2_mass))+ '\nChi2 mean of the source : ' + str(mean(ki2_source))+ 
      '\nChi2 mean of the mass & the source : ' +  str(mean(ki2_masssource))+ '\nChi2 mean of no error : ' + str(mean(ki2_noerror)))


sns.kdeplot(ki2_mass, label = 'Mass error', bw_adjust=.1)
sns.kdeplot(ki2_source, label = 'Source error', bw_adjust=.1)
sns.kdeplot(ki2_masssource, label = 'Mass & source error', bw_adjust=.1)
plt.xlim([0, 2])
plt.legend()
plt.savefig('figures/datanalysis/Chi2Noisefinal_chi2.jpeg')
plt.show()

In [ ]:
f, axes = plt.subplots(3, figsize=(10, 10), sharex='all', sharey='all',
                       gridspec_kw=dict(left=0.1, right=0.9,bottom=0.1, top=0.9))
percent = np.array([[0.005, 0.0142, 0.005], [0.005, 0.0142, 0.0049], [0.0074, 0.015, 0.0025]])
size = 6000; ratio = 0.75;

for ii in range(3):

    res = Residual()
    res.build(size, ratio = ratio, per_error = percent[ii,:])

    str_ID =  "S"+str(size)+"R"+str(int(ratio*100))
    [final_array, metadata] = read_hdf5(str_ID)

    print('-------------------------------------------------------------------------------------')

    index = metadata.index

    indices_noerror = index[[col == [0,0] for col in metadata['class']]]
    indices_mass = index[[col == [1,0] for col in metadata['class']]]
    indices_source = index[[col == [0,1] for col in metadata['class']]]
    indices_masssource = index[[col == [1,1] for col in metadata['class']]]

    dist_mass = [np.sum((final_array[indices_mass[i],0,:,:]-final_array[indices_noerror[i],0,:,:])**2)/64**2 for i in range(len(indices_noerror))]
    dist_source = [np.sum((final_array[indices_source[i],0,:,:]-final_array[indices_noerror[i],0,:,:])**2)/64**2 for i in range(len(indices_noerror))]
    dist_masssource = [np.sum((final_array[indices_masssource[i],0,:,:]-final_array[indices_noerror[i],0,:,:])**2)/64**2 for i in range(len(indices_noerror))]
    
    dens_mass = gaussian_kde(dist_mass)
    dens_source = gaussian_kde(dist_source)
    dens_masssource = gaussian_kde(dist_masssource)
    
    y_mass = dens_mass(np.linspace(1.5,5,200))
    y_source = dens_source(np.linspace(1.5,5,200))
    y_masssource = dens_masssource(np.linspace(1.5,5,200))

    print('Mean distance of the mass : '+ str(mean(dist_mass))+ '\nMean distance of the source : ' + str(mean(dist_source))+ 
          '\nMean distance of the mass & the source : ' +  str(mean(dist_masssource)))
    
    print('-------------------------')
    print('Max density distance of the mass : '+ str(y_mass[np.argmax(y_mass)])+ '\nMax density distance of the source : ' + str(y_source[np.argmax(y_source)])+ 
          '\nMax density distance of the mass & the source : ' +  str(y_masssource[np.argmax(y_masssource)]))
    
    
    sns.kdeplot(dist_mass, label = 'Mass error', bw_adjust=.1, ax=axes[ii])
    sns.kdeplot(dist_source, label = 'Source error', bw_adjust=.1, ax=axes[ii])
    sns.kdeplot(dist_masssource, label = 'Mass & source error', bw_adjust=.1, ax=axes[ii])
    axes[ii].set_xlim([1.5, 5])
    
axes[0].title.set_text('Chi2')
axes[1].title.set_text('Distance')
axes[2].title.set_text('Chi2 with noise consideration')
plt.legend()
plt.savefig('figures/datanalysis/distance_chi2.jpeg')
plt.show()

In [ ]:
f, axes = plt.subplots(3, figsize=(10, 10), sharex='all', sharey='all',
                       gridspec_kw=dict(left=0.1, right=0.9,bottom=0.1, top=0.9))
percent = np.array([[0.0074, 0.015, 0.0025], [0.005, 0.0142, 0.005], [0.005, 0.0142, 0.0049]])
ratio = 0.75; size = 600

for ii in range(3):

    res = Residual()
    res.build(size, ratio = ratio, per_error = percent[ii,:])

    str_ID =  "S"+str(size)+"R"+str(int(ratio*100))
    [final_array, metadata] = read_hdf5(str_ID)

    print('Reading data Finished')

    index = metadata.index

    indices_noerror = index[[col == [0,0] for col in metadata['class']]]
    indices_mass = index[[col == [1,0] for col in metadata['class']]]
    indices_source = index[[col == [0,1] for col in metadata['class']]]
    indices_masssource = index[[col == [1,1] for col in metadata['class']]]

    ki2_noerror = [np.sum(final_array[i,:,:]**2)/(final_array.shape[2]*final_array.shape[3])for i in indices_noerror]
    ki2_mass = [np.sum(final_array[i,:,:]**2)/(final_array.shape[2]*final_array.shape[3])for i in indices_mass]
    ki2_source = [np.sum(final_array[i,:,:]**2)/(final_array.shape[2]*final_array.shape[3])for i in indices_source]
    ki2_masssource = [np.sum(final_array[i,:,:]**2)/(final_array.shape[2]*final_array.shape[3])for i in indices_masssource]
    
    print('Chi2 mean of the mass : '+ str(mean(ki2_mass))+ '\nChi2 mean of the source : ' + str(mean(ki2_source))+ 
      '\nChi2 mean of the mass & the source : ' +  str(mean(ki2_masssource))+ '\nChi2 mean of no error : ' + str(mean(ki2_noerror)))
    
    axes[ii].hist(ki2_noerror)
    axes[ii].hist(ki2_mass)
    axes[ii].hist(ki2_source)
    axes[ii].hist(ki2_masssource)
    axes[ii].set_xlim([0, 2])
    
axes[0].title.set_text('Chi2 with noise consideration')
axes[1].title.set_text('Chi2')
axes[2].title.set_text('Distance')
plt.show()

In [ ]:
f, axes = plt.subplots(3, figsize=(10, 10), sharex='all', sharey='all',
                       gridspec_kw=dict(left=0.1, right=0.9,bottom=0.1, top=0.9))
percent = np.array([[0.0074, 0.015, 0.0025], [0.005, 0.0142, 0.005], [0.005, 0.0142, 0.0049]])
ratio = 0.75;

percent[0,:] = 0.8*np.array([0.0074, 0.015, 0.0025])
for ii in range(3):

    res = Residual()
    res.build(size, ratio = ratio, per_error = percent[ii,:])

    str_ID =  "S"+str(size)+"R"+str(int(ratio*100))
    [final_array, metadata] = read_hdf5(str_ID)

    print('Reading data Finished')

    index = metadata.index

    indices_noerror = index[[col == [0,0] for col in metadata['class']]]
    indices_mass = index[[col == [1,0] for col in metadata['class']]]
    indices_source = index[[col == [0,1] for col in metadata['class']]]
    indices_masssource = index[[col == [1,1] for col in metadata['class']]]

    rms_noise = np.asarray(res.rms_noise)
    noise_final = np.asarray([final_array[i,0,:,:] - 3*rms_noise[i,:,:] for i in range(size)])
    noise_final = noise_final.clip(min=0)

    ki2_noerror = [np.sum(noise_final[i,:,:]**2)/(final_array.shape[2]*final_array.shape[3])for i in indices_noerror]
    ki2_mass = [np.sum(noise_final[i,:,:]**2)/(final_array.shape[2]*final_array.shape[3])for i in indices_mass]
    ki2_source = [np.sum(noise_final[i,:,:]**2)/(final_array.shape[2]*final_array.shape[3])for i in indices_source]
    ki2_masssource = [np.sum(noise_final[i,:,:]**2)/(final_array.shape[2]*final_array.shape[3])for i in indices_masssource]

    print('Chi2 mean of the mass : '+ str(mean(ki2_mass))+ '\nChi2 mean of the source : ' + str(mean(ki2_source))+ 
          '\nChi2 mean of the mass & the source : ' +  str(mean(ki2_masssource))+ '\nChi2 mean of no error : ' + str(mean(ki2_noerror)))
    
    sns.kdeplot(ki2_mass, label = 'Mass error', bw_adjust=.1, ax=axes[ii])
    sns.kdeplot(ki2_source, label = 'Source error', bw_adjust=.1, ax=axes[ii])
    sns.kdeplot(ki2_masssource, label = 'Mass & source error', bw_adjust=.1, ax=axes[ii])
    axes[ii].set_xlim([0, 2])
    
axes[0].title.set_text('Chi2 with noise consideration')
axes[1].title.set_text('Chi2')
axes[2].title.set_text('Distance')
plt.legend()
plt.savefig('figures/datanalysis/Chi2Noise_chi2.jpeg')
plt.show()